In [1]:
%load_ext autotime

time: 134 µs (started: 2023-07-18 17:35:31 +02:00)


In [2]:
import warnings
warnings.filterwarnings('ignore', message="Casting complex values to real discards the imaginary part")

time: 386 µs (started: 2023-07-18 17:35:31 +02:00)


In [3]:
from joblib import Parallel, delayed
import multiprocessing

time: 505 ms (started: 2023-07-18 17:35:32 +02:00)


In [4]:
import json
import random
from mei import Size_Distribution_Optics, MieCoated, Mie
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

time: 2.37 s (started: 2023-07-18 17:35:34 +02:00)


In [5]:
num_cores = multiprocessing.cpu_count()
num_cores

256

time: 2.8 ms (started: 2023-07-18 17:35:38 +02:00)


In [6]:
# Load refractive indices
def read_refractive_indices(filename, columns=["lam", "n", "k"]):
    data = pd.read_csv(
        filename,
        sep="\s+",
        names=columns,
    )
    return data


def interpolate_indices(B_blc1, B_brc1):
    f1 = interp1d(B_blc1.lam, B_blc1["n"])
    f2 = interp1d(B_blc1.lam, B_blc1["k"])

    new1 = pd.DataFrame([])
    new1["lam"] = B_brc1.lam
    new1["n"] = f1(B_brc1.lam)
    new1["k"] = f2(B_brc1.lam)
    return new1


def model_data(new, model="SEVIRI"):
    if model == "ICON":
        wavel_interp = [
            210,
            220,
            230,
            240,
            3900,
            4100,
            4200,
            4250,
            4300,
            4350,
            4400,
            4600,
            4700,
            4800,
            4900,
            5100,
            5200,
            5300,
            5400,
            6800,
            6900,
            7000,
            7100,
            14400,
            15400,
            50000,
            60000,
            75000,
            90000,
        ]
        lam = list(new.lam.values)        
        wavel_interp = list(set([*wavel_interp, *lam]))

    elif model == "SEVIRI":
        wavel_interp = [
            637.39,
            807.06,
            1635.65,
            3913.47,
            6265.35,
            7343.50,
            8715.11,
            9663.23,
            10739.73,
            11917.35,
            13358.54,
            612.19,
        ]

    elif model == "AERONET":
        wavel_interp = [340, 380, 440, 500, 550, 675, 870, 1020, 1064]

    else:
        print("Please enter a correct model.")

    n_band = len(wavel_interp)

    # make sure wavelenghts are in ascending order
    wavel_interp = np.sort(wavel_interp)

    f1 = interp1d(new.lam, new["n"], fill_value="extrapolate", kind="linear")
    f2 = interp1d(new.lam, new["k"], fill_value="extrapolate", kind="linear")

    new1 = pd.DataFrame([])
    new1["lam"] = wavel_interp
    new1["n"] = f1(wavel_interp)
    new1["k"] = f2(wavel_interp)

    return new1

time: 1.11 ms (started: 2023-07-18 17:35:40 +02:00)


In [7]:
def make_serializable(opt):
    for key in opt:
        if isinstance(opt[key], np.ndarray):
            opt[key] = opt[key].tolist()
        if isinstance(opt[key], np.int32):
            opt[key] = int(opt[key])
    return opt


def save_opt(data, filename):
    with open(filename, "w") as fp:
        json.dump(data, fp)


def load_opt(filename):
    with open(filename, "r") as fp:
        data = json.load(fp)
    return data

time: 545 µs (started: 2023-07-18 17:35:44 +02:00)


In [8]:
def run_mie_bin(df, index, outpath):
    dx = df.iloc[index, :]
    m_shell = dx['n_shell'] + dx['k_shell'] * 1j
    m_core = dx['n_core'] + dx['k_core'] * 1j

    mr = 1
    mc = m_shell
    mp = m_core

    coating = dx['coating']
    iscoated = coating != 0
    
    const = np.pi * mr / dx['lam']

    xval =  const * dx['bin']
    yval = xval * (1 + coating)

    if iscoated:
        #print('Miecoated called')
        one_result = MieCoated(mp / mr, mc / mr, xval, yval)
    else:
        one_result = Mie(mp / mr, xval)

    Mie_tots = pd.Series([], dtype="float")

    Mie_tots["Extinction"] = one_result[0]
    Mie_tots["Scattering"] = one_result[1]
    Mie_tots["Absorption"] = one_result[2]
    Mie_tots["Asym"] = one_result[4]
    Mie_tots["SSA"] = Mie_tots["Scattering"] / Mie_tots["Extinction"]
    Mie_tots["coating"] = coating
    Mie_tots["core_dia"] = dx['bin']


    Mie_tots["lambda"] = dx['lam']
    Mie_tots["n_core"] = dx['n_core']
    Mie_tots["k_core"] = dx['k_core']
    Mie_tots["n_shell"] = dx['n_shell']
    Mie_tots["k_shell"] = dx['k_shell']
    Mie_tots["density"] = dx['rho']
    Mie_tots["nobackscat"] = True

    Mie_tots.to_csv('%s/mie_%s.csv'%(outpath, index), index=True)
    return None

time: 759 µs (started: 2023-07-18 17:35:45 +02:00)


In [9]:
def get_parallel_list(rand_idx, num_cores):
    num = len(np.array_split(rand_idx, num_cores-5, axis=0)[0])
    new_list = np.array_split(rand_idx, num, axis=0)
    return new_list    

time: 260 µs (started: 2023-07-18 17:35:46 +02:00)


In [10]:
model = "ICON"

time: 141 µs (started: 2023-07-18 17:35:47 +02:00)


In [11]:
ri_sol = np.load("../output/ri_sol.npy")
rho_sol = np.load("../output/rho_sol.npy")

ri_liq = np.load("../output/ri_liq.npy")
rho_liq = np.load("../output/rho_liq.npy")

wavel_interp = np.load("../output/wavel_interp.npy")
ri_sol.shape, rho_sol.shape, ri_liq.shape, rho_liq.shape, wavel_interp.shape

((30, 37, 2), (30,), (30, 37, 2), (30,), (37,))

time: 38.6 ms (started: 2023-07-18 17:35:48 +02:00)


In [12]:
bin1 = list(range(10, 100, 10))

bin2 = list(np.arange(100, 1000, 100))

bin3 = list(np.arange(1000, 20001, 1000))

bins = [*bin1, *bin2, *bin3]

nsurround = 1

res = 20
fontsize = 20

time: 743 µs (started: 2023-07-18 17:35:50 +02:00)


In [13]:
count = 0
df = []

for coating in np.arange(0, 0.41, 0.02):
    for jk in np.arange(len(rho_liq)):
        rho1 = rho_liq[jk]

        new = pd.DataFrame([])
        new["lam"] = wavel_interp * 1000
        new["n"] = ri_liq[jk, :, 0]
        new["k"] = ri_liq[jk, :, 1]

        new2 = model_data(new, model=model)

        for ik in np.arange(len(rho_sol)):
            rho = rho_sol[ik]

            new = pd.DataFrame([])
            new["lam"] = wavel_interp * 1000
            new["n"] = ri_sol[ik, :, 0]
            new["k"] = ri_sol[ik, :, 1]

            new1 = model_data(new, model=model)
            
            for i, bn in enumerate(bins):
                for lam, n, k, n1, k1 in zip(new1.lam, new1.n, new1.k, new2.n, new2.k):
                    count += 1
                    df.append([rho, bn, lam, n, k, n1, k1, coating])            
            
df = pd.DataFrame(df, columns = ['rho', 'bin', 'lam', 'n_core', 'k_core', 'n_shell', 'k_shell', 'coating'])
# df = df.iloc[:10, :]
# df.to_csv('./output/mie_data.csv', index=False)
df

,rho,bin,lam,n_core,k_core,n_shell,k_shell,coating
0,2.6,10,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0
1,2.6,10,150.0,1.4860,0.106100,1.4300,1.865000e-07,0.0
2,2.6,10,200.0,1.4860,0.106100,1.3960,1.100000e-07,0.0
3,2.6,10,210.0,1.4837,0.099080,1.3892,9.470000e-08,0.0
4,2.6,10,220.0,1.4814,0.092060,1.3824,7.940000e-08,0.0
...,...,...,...,...,...,...,...,...
44528395,1.7,20000,70000.0,1.6135,0.648098,2.0150,7.250000e-01,0.4
44528396,1.7,20000,75000.0,1.6135,0.648098,2.0175,7.225000e-01,0.4
44528397,1.7,20000,80000.0,1.6135,0.648098,2.0200,7.200000e-01,0.4
44528398,1.7,20000,90000.0,1.6135,0.648098,2.0200,6.850000e-01,0.4


time: 3min 5s (started: 2023-07-18 17:35:50 +02:00)


In [14]:
# df = pd.read_csv('./output/mie_data.csv')

time: 129 µs (started: 2023-07-18 17:38:56 +02:00)


In [15]:
df['x'] = np.pi * df['bin'] / df['lam']
df

,rho,bin,lam,n_core,k_core,n_shell,k_shell,coating,x
0,2.6,10,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0,0.314159
1,2.6,10,150.0,1.4860,0.106100,1.4300,1.865000e-07,0.0,0.209440
2,2.6,10,200.0,1.4860,0.106100,1.3960,1.100000e-07,0.0,0.157080
3,2.6,10,210.0,1.4837,0.099080,1.3892,9.470000e-08,0.0,0.149600
4,2.6,10,220.0,1.4814,0.092060,1.3824,7.940000e-08,0.0,0.142800
...,...,...,...,...,...,...,...,...,...
44528395,1.7,20000,70000.0,1.6135,0.648098,2.0150,7.250000e-01,0.4,0.897598
44528396,1.7,20000,75000.0,1.6135,0.648098,2.0175,7.225000e-01,0.4,0.837758
44528397,1.7,20000,80000.0,1.6135,0.648098,2.0200,7.200000e-01,0.4,0.785398
44528398,1.7,20000,90000.0,1.6135,0.648098,2.0200,6.850000e-01,0.4,0.698132


time: 373 ms (started: 2023-07-18 17:38:56 +02:00)


In [16]:
df1 = df[df.x > 0.5]
df1

,rho,bin,lam,n_core,k_core,n_shell,k_shell,coating,x
62,2.6,20,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0,0.628319
124,2.6,30,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0,0.942478
125,2.6,30,150.0,1.4860,0.106100,1.4300,1.865000e-07,0.0,0.628319
186,2.6,40,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0,1.256637
187,2.6,40,150.0,1.4860,0.106100,1.4300,1.865000e-07,0.0,0.837758
...,...,...,...,...,...,...,...,...,...
44528395,1.7,20000,70000.0,1.6135,0.648098,2.0150,7.250000e-01,0.4,0.897598
44528396,1.7,20000,75000.0,1.6135,0.648098,2.0175,7.225000e-01,0.4,0.837758
44528397,1.7,20000,80000.0,1.6135,0.648098,2.0200,7.200000e-01,0.4,0.785398
44528398,1.7,20000,90000.0,1.6135,0.648098,2.0200,6.850000e-01,0.4,0.698132


time: 3.03 s (started: 2023-07-18 17:38:56 +02:00)


In [17]:
df2 = df[df.x <= 0.5]
df2

,rho,bin,lam,n_core,k_core,n_shell,k_shell,coating,x
0,2.6,10,100.0,1.4860,0.106100,1.4640,2.630000e-07,0.0,0.314159
1,2.6,10,150.0,1.4860,0.106100,1.4300,1.865000e-07,0.0,0.209440
2,2.6,10,200.0,1.4860,0.106100,1.3960,1.100000e-07,0.0,0.157080
3,2.6,10,210.0,1.4837,0.099080,1.3892,9.470000e-08,0.0,0.149600
4,2.6,10,220.0,1.4814,0.092060,1.3824,7.940000e-08,0.0,0.142800
...,...,...,...,...,...,...,...,...,...
44527964,1.7,13000,90000.0,1.6135,0.648098,2.0200,6.850000e-01,0.4,0.453786
44527965,1.7,13000,100000.0,1.6135,0.648098,2.0200,6.500000e-01,0.4,0.408407
44528026,1.7,14000,90000.0,1.6135,0.648098,2.0200,6.850000e-01,0.4,0.488692
44528027,1.7,14000,100000.0,1.6135,0.648098,2.0200,6.500000e-01,0.4,0.439823


time: 767 ms (started: 2023-07-18 17:38:59 +02:00)


In [18]:
df.iloc[100000, :]

rho            2.150000
bin          800.000000
lam        60000.000000
n_core         2.123381
k_core         0.341212
n_shell        1.568100
k_shell        0.419500
coating        0.000000
x              0.041888
Name: 100000, dtype: float64

time: 1.81 ms (started: 2023-07-18 17:39:00 +02:00)


In [33]:
frac = 0.2
ln1 = int(len(df1) * frac)
ln2 = int(len(df2) * frac)
ln1, ln2

(5488560, 3417120)

time: 5.68 ms (started: 2023-07-18 17:45:45 +02:00)


In [34]:
np.random.seed(42)
rand_idx1 = np.sort(np.random.choice(df1.index.values, 6000000, replace=False))
rand_idx2 = np.sort(np.random.choice(df2.index.values, 4000000, replace=False))

time: 2.38 s (started: 2023-07-18 17:45:46 +02:00)


In [35]:
rand_idx1

array([     192,      249,      254, ..., 44528394, 44528395, 44528399])

time: 1.24 ms (started: 2023-07-18 17:45:49 +02:00)


In [36]:
rand_idx2

array([       0,        2,        8, ..., 44527839, 44527902, 44527903])

time: 939 µs (started: 2023-07-18 17:45:50 +02:00)


In [37]:
new_list1 = get_parallel_list(rand_idx1, num_cores)
new_list2 = get_parallel_list(rand_idx2, num_cores)
# new_list1

time: 39 ms (started: 2023-07-18 17:45:51 +02:00)


In [38]:
len(new_list1), len(new_list2), len(new_list2[2]),

(23905, 15937, 251)

time: 943 µs (started: 2023-07-18 17:45:52 +02:00)


In [ ]:
# total batches 21585 processed for list 1

In [ ]:
#for i, inputs in enumerate(new_list1):
#    print('Working on batch: %d'%i)
#    Parallel(n_jobs=num_cores)(delayed(run_mie_bin)(df, index, outpath='/work/bb1070/b382177/mie/icon/18-07-2023/x1') for index in inputs)

Working on batch: 0
Working on batch: 1
Working on batch: 2
Working on batch: 3
Working on batch: 4
Working on batch: 5
Working on batch: 6
Working on batch: 7
Working on batch: 8
Working on batch: 9
Working on batch: 10
Working on batch: 11
Working on batch: 12
Working on batch: 13
Working on batch: 14
Working on batch: 15
Working on batch: 16
Working on batch: 17
Working on batch: 18
Working on batch: 19
Working on batch: 20
Working on batch: 21
Working on batch: 22
Working on batch: 23
Working on batch: 24
Working on batch: 25
Working on batch: 26
Working on batch: 27
Working on batch: 28
Working on batch: 29
Working on batch: 30
Working on batch: 31
Working on batch: 32
Working on batch: 33
Working on batch: 34
Working on batch: 35
Working on batch: 36
Working on batch: 37
Working on batch: 38
Working on batch: 39
Working on batch: 40
Working on batch: 41
Working on batch: 42
Working on batch: 43
Working on batch: 44
Working on batch: 45
Working on batch: 46
Working on batch: 47
Wo

In [39]:
for i, inputs in enumerate(new_list2):
    print('Working on batch: %d'%i)
    Parallel(n_jobs=num_cores)(delayed(run_mie_bin)(df, index, outpath='/work/bb1070/b382177/mie/icon/18-07-2023/x2') for index in inputs)

Working on batch: 0
Working on batch: 1
Working on batch: 2
Working on batch: 3
Working on batch: 4
Working on batch: 5
Working on batch: 6
Working on batch: 7
Working on batch: 8
Working on batch: 9
Working on batch: 10
Working on batch: 11
Working on batch: 12
Working on batch: 13
Working on batch: 14
Working on batch: 15
Working on batch: 16
Working on batch: 17
Working on batch: 18
Working on batch: 19
Working on batch: 20
Working on batch: 21
Working on batch: 22
Working on batch: 23
Working on batch: 24
Working on batch: 25
Working on batch: 26
Working on batch: 27
Working on batch: 28
Working on batch: 29
Working on batch: 30
Working on batch: 31
Working on batch: 32
Working on batch: 33
Working on batch: 34
Working on batch: 35
Working on batch: 36
Working on batch: 37
Working on batch: 38
Working on batch: 39
Working on batch: 40
Working on batch: 41
Working on batch: 42
Working on batch: 43
Working on batch: 44
Working on batch: 45
Working on batch: 46
Working on batch: 47
Wo

In [ ]:
pd.read_csv('/work/bb1070/b382177/mie/icon/18-07-2023/x2/mie_%s.csv'%rand_idx2[-1], index_col=0)

,0
Extinction,0.6712109240212665
Scattering,0.0687112889884546
Absorption,0.6024996350328119
Asym,0.06785948546632262
SSA,0.10236914586669864
coating,0.4
core_dia,12000.0
lambda,100000.0
n_core,1.6135
k_core,0.64809787


time: 26.1 ms (started: 2023-07-18 23:23:51 +02:00)
